In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
import os
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences

C:\Users\Andre\Anaconda3\envs\GreatEnv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
inputPath = os.path.join(os.getcwd(),"input")
print(os.listdir(inputPath))

['sample_submission.csv', 'test.csv', 'train.csv']


In [12]:
pd.read_csv(os.path.join(inputPath,"sample_submission.csv")).head()

,item_id,deal_probability
0,6544e41a8817,0.5
1,65b9484d670f,0.5
2,8bab230b2ecd,0.5
3,8e348601fefc,0.5
4,8bd2fe400b89,0.5


In [6]:
trainDF = pd.read_csv(os.path.join(inputPath,"train.csv"))
testDF = pd.read_csv(os.path.join(inputPath,"test.csv"))

trainDF.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [7]:
trainDF = trainDF[["description","price","deal_probability"]].dropna(axis=0, how='any')
testDF = testDF[["item_id","price","description"]].fillna(0)

max_features = 20000
maxlen = 200

tokenizer = Tokenizer(num_words=max_features)

tokenizer.fit_on_texts(trainDF["description"])

trainDF["description"] = tokenizer.texts_to_sequences(trainDF["description"])
testDF["description"]  = tokenizer.texts_to_sequences(testDF["description"])

train_x = pad_sequences(trainDF["description"], maxlen=maxlen)
test_x = pad_sequences(testDF["description"], maxlen=maxlen)

#trainDF['category_name'], mapping_index_category = pd.Series(trainDF['category_name']).factorize()
#testDF['category_name'], mapping_index_category2 = pd.Series(resDF['category_name']).factorize()

#resDF['category_name'] = resDF['category_name'].astype('category')
#resDF['category_name'], mapping_index_category = pd.Series(resDF['category_name']).factorize()
#resDF['city'], mapping_index_city = pd.Series(resDF['city']).factorize()
#resDF = resDF.dropna(axis=0, how='any')
#resDF["description"] = resDF["description"].map(lambda z: len(z))
#resDF[["price"]] /= resDF[["price"]].max()
#resDF[["price","category_name","description"]] /= resDF[["price","category_name","description"]].max()
#plt.matshow(trainDF[["price","category_name","city","deal_probability"]].corr())
#plt.show()
#plt.subplots(figsize=(10,10))
#corr = resDF.corr()
#sns.heatmap(corr, 
#            xticklabels=corr.columns.values,
#            yticklabels=corr.columns.values,)

In [8]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,Concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

embed_size = 128
inp = Input(shape=(maxlen, )) #maxlen=200 as defined earlier
x = Embedding(max_features, embed_size)(inp)
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)

priceInp = Input(shape=(1, ))
x = Concatenate(axis=-1)([priceInp,x])

x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)

model = Model(inputs=[inp,priceInp], outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])


In [9]:
model.fit([train_x,trainDF["price"]] ,trainDF[["deal_probability"]],epochs=5, batch_size=100,validation_split=0.1)

Train on 1174813 samples, validate on 130535 samples
Epoch 1/5
1174813/1174813 [==============================] - 2762s 2ms/step - loss: 1.0010 - acc: 0.6448 - val_loss: 0.3838 - val_acc: 0.6505
Epoch 2/5
1174813/1174813 [==============================] - 2760s 2ms/step - loss: 0.3829 - acc: 0.6503 - val_loss: 0.3791 - val_acc: 0.6505
Epoch 3/5
1174813/1174813 [==============================] - 2760s 2ms/step - loss: 0.3729 - acc: 0.6504 - val_loss: 0.3713 - val_acc: 0.6507
Epoch 4/5
1174813/1174813 [==============================] - 2761s 2ms/step - loss: 0.3675 - acc: 0.6504 - val_loss: 0.3668 - val_acc: 0.6507
Epoch 5/5
1174813/1174813 [==============================] - 2756s 2ms/step - loss: 0.3669 - acc: 0.6503 - val_loss: 0.3694 - val_acc: 0.6499


In [ ]:
modelPred = model.predict([test_x,testDF["price"]],batch_size=300).reshape(len(test_x))

In [ ]:
data_to_submit = pd.DataFrame.from_items([
    ('item_id',testDF["item_id"]),
    ('deal_probability', pd.Series(modelPred))])

data_to_submit.to_csv('csv_to_submit.csv', index = False)